# Loading the libraries

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-class-path /usr/share/java/postgresql-42.2.23.jar --jars /usr/share/java/postgresql-42.2.23.jar pyspark-shell'
import databricks.koalas as ks
import pandas as pd
import networkx as nx
from heapq import heapify, heappop, heappush

# Splitting the lengthy edges

In [ ]:
def splitUntilDouble(G):
    firstSplit = 2
    edgesHeap = [(-1*data['length']/firstSplit, u, v, data['idedge'], firstSplit) for u, v, data in G.edges(data=True)]
    heapify(edgesHeap)

    for i in range(len(edgesHeap)):
        (length, u, v, idedge, numSplit) = heappop(edgesHeap)
        heappush(edgesHeap, (length * numSplit/(numSplit+1), u, v, idedge))
    
    for item in edgesHeap:
        numSplit = item[-1]
        if numSplit > firstSplit:
            (length, u, v, idedge, numSplit) = item
            